# What is about ?

Example how to load scRNA data and very first simple look on data.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import and install modules

In [ ]:
!pip install scanpy
import scanpy as sc
import anndata

import scipy 


import time
t0start = time.time()

import pandas as pd
import numpy as np
import os
import sys

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 70
plt.style.use('dark_background')

import seaborn as sns

from sklearn.decomposition import PCA

# Load data 

In [ ]:
def read_mtx_by_prefix(path_and_fn_prefix):
    adata = sc.read(path_and_fn_prefix + 'matrix.mtx').T
    genes = pd.read_csv(path_and_fn_prefix + 'genes.tsv', header=None, sep='\t') 
    genes = genes.set_index(1)
    genes.index.name = 'Gene'
    genes.columns = ['Ensemble']
    adata.var = genes
    
    barcodes = pd.read_csv(path_and_fn_prefix + 'barcodes.tsv', header=None, sep='\t', index_col = 0) 
    barcodes.index.name = 'barcodes'
    
    adata.obs = barcodes

    return adata

if 1:
    str_data_inf = ' GSE64016_human_H1hESC_Fucci_Y2014_460Cells_GEO_counts_Leng_Kendziorski '
    path_and_fn = '/kaggle/input/scrnaseq-hesc-h1-cell-line-fucci-cell-cycle-label/GSE64016_human_H1hESC_Fucci_Y2014_460Cells_GEO_counts_Leng_Kendziorski.h5ad'
    adata_orig = sc.read(path_and_fn)
else:    
    str_data_inf = ' GSM3267241_U5_all '
    path_and_fn_prefix = '/kaggle/input/scrnaseq-neuroepithelialderived-cells-and-glioma/GSM3267241_U5_all_'
    adata_orig =  read_mtx_by_prefix(path_and_fn_prefix)
    
    
adata = adata_orig.copy()
adata_orig


# EDA

## Show top expressed genes

In [ ]:
v = np.asarray(adata.X.sum(axis = 0)).ravel()
adata.var['Expression Sum'] = v
v = np.asarray( (adata.X != 0) .sum(axis = 0)).ravel()
adata.var['Count Expressed Cells'] = v
adata.var.sort_values('Expression Sum',ascending = False ).head(30) # ['counts']

## Expression per gene

In [ ]:
v = np.asarray(adata.X.sum(axis = 0)).ravel()
display(pd.Series(v).describe())
v = np.log10(1+v)
fig = plt.figure(figsize = (20,6)); c = 0
c+=1; fig.add_subplot(1,2,c);
plt.plot(np.sort(v)) 
plt.title('LOG10 Expression per gene')
plt.xlabel('genes sorted')
plt.ylabel('Log10 (1+Counts) ')

c+=1; fig.add_subplot(1,2,c);
plt.hist(np.sort(v), bins = 30) 
plt.title('LOG10 Expression per gene')

plt.show()

pd.Series(v).describe()

## Expression per cell

In [ ]:
fig = plt.figure(figsize = (20,6)); c = 0
c+=1; fig.add_subplot(1,2,c);
v = np.asarray(adata.X.sum(axis = 1)).ravel() # adata.obs['n.umi']
plt.plot(np.sort(v ),'*-') 
plt.title('Expression per cell')
plt.xlabel('cells sorted')
plt.ylabel('Expression Total')

c+=1; fig.add_subplot(1,2,c);
plt.hist(np.sort(v), bins = 100) 
plt.title('Expression per cell')

plt.show()

pd.Series(v).describe()


fig = plt.figure(figsize = (20,6)); c = 0
c+=1; fig.add_subplot(1,2,c);
v = np.asarray((adata.X != 0).sum(axis = 1)).ravel() # adata.obs['n.umi']
plt.plot(np.sort(v ),'*-') 
plt.title('Count genes expressed per cell')
plt.xlabel('cells sorted')
plt.ylabel('Count expressed genes')

c+=1; fig.add_subplot(1,2,c);
plt.hist(np.sort(v), bins = 100) 
plt.title('Count genes expressed per cell')

plt.show()

pd.Series(v).describe()

# Correlation analysis

In [ ]:
import scipy 

number_top_genes_to_consider = 10_000
v = np.asarray(adata.X.sum(axis = 0 ) ).ravel()
v.shape
I = np.argsort(v)
I = I[::-1]
I = I[:number_top_genes_to_consider]

mask = np.ones( adata.X.shape[0]).astype(bool) 
t0 = time.time()
if scipy.sparse.issparse(adata.X):
    corr_matr = np.corrcoef(adata[mask].X[:,I].toarray().T) # Hint - use numpy , pandas is MUCH SLOWER   (df.corr() )
else:
    corr_matr = np.corrcoef(adata[mask].X[:,I].T) # Hint - use numpy , pandas is MUCH SLOWER   (df.corr() )
print('Calculated correlation matrix. Shape: ',corr_matr.shape,'  seconds passed:' , np.round( time.time() - t0,1) )

print()
print(np.min(corr_matr ), 'minimal correlation' )
corr_matr_abs = np.abs( corr_matr )
print(np.mean(corr_matr_abs ), 'average absolute correlation' )
print(np.median(corr_matr_abs), 'median absolute correlation' )
print(np.min(corr_matr_abs ), 'min absolute correlation' )
print(np.std(corr_matr_abs ), 'std absolute correlation' )
print()

v = corr_matr.flatten()
plt.figure(figsize=(14,8))
t0 = time.time()
plt.hist(v, bins = 50)
plt.title('correlation coefficients distribution')
plt.show()
print(time.time() - t0, 'seconds passed')

print(np.min(corr_matr ), 'minimal correlation' )
print(np.mean(corr_matr_abs ), 'average absolute correlation' )
print(np.median(corr_matr_abs), 'median absolute correlation' )
print(np.min(corr_matr_abs ), 'min absolute correlation' )
print(np.std(corr_matr_abs ), 'std absolute correlation' )
for t in [0.5,0.6, 0.7,0.8,0.9,0.95,0.97,0.98,.99]:
    print( ((np.abs(v) < 0.99999999) & (np.abs(v) > t)).sum()/2 , 'number of pairs correlated more than', t  )
v.shape

print()
print()
for threshold_corr in [0.99,0.98,00.97,0.95,0.93,0.90,0.85,0.8,0.75,0.7,0.65,0.6,0.55,.5,0.45,0.4,0.35,0.3]: 
    a,b = np.where( np.triu(np.abs(corr_matr),1) > threshold_corr )
    if len(a) > 50: break
print('for threshold:', threshold_corr,'we have ',len(a),' correlated pairs' )


d = pd.DataFrame()
for i in range(len(a)):
    Iai, Ibi =  I[a[i]], I[b[i]]
    d.loc[i,'Gene1'] = adata[mask].var.index[Iai] 
    d.loc[i,'Gene2'] = adata[mask].var.index[Ibi ] 
    d.loc[i,'Correlation'] = corr_matr[a[i],b[i]] 
    d.loc[i,'Correlation Abs'] = np.abs(d.loc[i,'Correlation'] )
    d.loc[i,'Gene1 Expression Sum'] = np.asarray(adata[mask].X[:,Iai].sum(axis = 0)).ravel()[0]
    d.loc[i,'Gene2 Expression Sum'] = np.asarray(adata[mask].X[:,Ibi].sum(axis = 0)).ravel()[0]
    
print(time.time() - t0, 'seconds passed')
    
d.sort_values('Correlation Abs',ascending = False).head(30)

In [ ]:
import seaborn as sns
t0 = time.time()
sns.clustermap(np.abs(corr_matr[:3000,:3000]),cmap='vlag');
print( np.round(time.time()- t0,1), ' seconds passed.')

# Visualisation


## PCA


In [ ]:
import scipy
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
import time
t0 = time.time()
if scipy.sparse.issparse(adata.X):
    reducer = TruncatedSVD(n_components=2, n_iter=7, random_state=42)
else:
    reducer = PCA(n_components=2)
    
r = reducer.fit_transform(adata.X)

fig = plt.figure(figsize = (25,12))
plt.title(str_data_inf + ' PCA  n_cells: ' + str(adata.X.shape[0]) +\
            ' n_genes: ' + str(adata.X.shape[1])  + ' '  , fontsize = 20   )#' 

if 'n.umi' not in adata.obs.columns: 
    ax = sns.scatterplot(x=r[:,0], y = r[:,1])#,  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
else:
    color_by = adata.obs['n.umi'].values
    ax = sns.scatterplot(x=r[:,0], y = r[:,1],  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
    plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title
plt.xlabel('PCA1' , fontsize = 20 )
plt.ylabel('PCA2' , fontsize = 20 )
plt.show()
print( np.round( time.time() - t0,1) , ' seconds passed ' )

## UMAP

In [ ]:
import scipy
import umap 
t0 = time.time()
reducer = umap.UMAP()
r = reducer.fit_transform(adata.X)

fig = plt.figure(figsize = (25,12))
plt.title(str_data_inf + ' UMAP n_cells: ' + str(adata.X.shape[0]) +\
            ' n_genes: ' + str(adata.X.shape[1])  + ' '  , fontsize = 20   )#' 
if 'n.umi' not in adata.obs.columns: 
    ax = sns.scatterplot(x=r[:,0], y = r[:,1])#,  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
else:
    color_by = adata.obs['n.umi'].values
    ax = sns.scatterplot(x=r[:,0], y = r[:,1],  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
    plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title

plt.xlabel('UMAP1' , fontsize = 20 )
plt.ylabel('UMAP2' , fontsize = 20 )
plt.show()
print( np.round( time.time() - t0,1) , ' seconds passed ' ) 